In [ ]:
%pip install pandas langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt langchain_upstage

In [2]:
import os 
import dotenv 

dotenv.load_dotenv()

True

In [3]:
import os
import pandas as pd
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from pprint import pprint

# GPT_API_KEY = os.environ.get("OPENAI_API_KEY")
UPSTAGE_API_KEY = os.environ.get("UPSTAGE_API_KEY")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

In [3]:
folder_path = '../data'

In [ ]:
document_list = []

for file in os.listdir(folder_path):
    print(file)
    temp_loader = CSVLoader(file_path=f"{folder_path}/{file}", encoding='utf-8-sig')
    temp_document_list = temp_loader.load_and_split(text_splitter=text_splitter)
    
    document_list.extend(temp_document_list)

print(len(document_list))

In [4]:
# Upstage 에서 제공하는 Embedding Model을 활용
embedding = UpstageEmbeddings(model="solar-embedding-1-large",
                              api_key=UPSTAGE_API_KEY)

In [5]:
from langchain_pinecone import PineconeVectorStore

index_name = 'upstage-index'

In [ ]:
# DB 처음 만들 때
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [6]:
# 만들어 놓은 DB가 있을 때
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

# Vectorstore 유사도 검색

In [47]:
query = ('결제 시 카드정보는 어떻게 보호되나요?')

results = database.similarity_search_with_score(query=query, k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")
    

* [SIM=0.383785] : 67
Question: Q[결제수단] 인터넷안전결제(ISP) 서비스란 무엇인가요?
Answer: 인터넷안전결제(ISP)란 카드 결제 시 카드번호를 직접 입력하지 않고 카드사에서 발급하는 가상의 인터넷안전결제(ISP) 비밀번호만으로 안전한 전자상거래를 할 수 있는 서비스입니다. 단 거래 금액이 30만원 이상일 때에는 공인 인증 과정이 필요합니다. 카드사별로 인터넷안전결제(ISP) 지원 여부가 상이하므로 자세한 내용은 사용하는 카드사로 문의해 주시기 바랍니다.
keywords: ORDER
count: 0 [{'row': 67.0, 'source': '../data/ORDER_main_qna_list.csv'}]
* [SIM=0.364467] : 95
Question: Q[결제] 토스카드/카카오페이 카드는 어떻게 결제하나요?
Answer: 신용/체크카드에서 ‘BC카드’를 선택한 후 [결제하기]를 클릭하면 카드 정보를 입력하여 결제할 수 있습니다.
 단 토스유스카드는 일반 결제 시 ‘KB국민카드’로 선택하여 결제할 수 있습니다.
keywords: ORDER
count: 0 [{'row': 95.0, 'source': '../data/ORDER_main_qna_list.csv'}]
* [SIM=0.363860] : 66
Question: Q[안심클릭] 안심클릭 서비스란 무엇인가요?
Answer: 인터넷 쇼핑 시 고객님의 카드에 미리 설정한 전자상거래용 별도 비밀번호를 입력하여 카드 사용자 본인을 확인함으로써 온라인상에서 카드의 무단 도용을 방지하는 서비스입니다. 카드사별로 이용 방법이 상이하므로 자세한 내용은 사용하는 카드사로 문의해 주시기 바랍니다.
keywords: ORDER
count: 0 [{'row': 66.0, 'source': '../data/ORDER_main_qna_list.csv'}]


# LLM 질의 테스트

In [ ]:
%pip install langchain_google_genai

In [48]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# ChatGoogleGenerativeAI 언어 모델을 초기화합니다.
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",  # 사용할 모델을 지정합니다.
)

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.

또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.
쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

만약, 조건별로 안내 내용이 다른 경우
1차 응대 (양해멘트 or 1차 안내 등) + 정보 확인 멘트로 대본을 구성하면 됩니다.
정보 확인 멘트는 "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다." 입니다.
문서의 아래에 각 조건별 대응 방법을 기술해 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.3보다 낮게 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
%pip install langchain_ollama

In [49]:
print("query: ", query)
print("=" * 80)
answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:
    print(ans + '.')

query:  결제 시 카드정보는 어떻게 보호되나요?
Answer : 고객님, 결제 시 카드 정보 보호에 대해 문의주셨네요.
 안전한 결제를 위해 최선을 다하고 있습니다.
 우선, 인터넷안전결제(ISP) 서비스를 이용하시면 카드번호를 직접 입력하지 않고 가상 비밀번호로 결제하실 수 있습니다.
 단, 30만원 이상 결제 시에는 공인인증서가 필요할 수 있으니 참고 부탁드립니다.
 토스카드나 카카오페이 카드는 BC카드 또는 KB국민카드로 결제 가능하며, 쿠페이는 별도의 공인인증서나 보안카드 없이 비밀번호만으로 결제 가능합니다.
 더 자세한 내용은 사용하시는 카드사에 문의하시는 것이 좋습니다.



**조건별 대응 방법:**

(해당 질문은 제공된 context 내 정보만으로 충분히 답변 가능하며, 추가적인 정보 확인이 필요하지 않습니다.
)
.


#### Ollama 버전 

In [40]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

# ChatGoogleGenerativeAI 언어 모델을 초기화합니다.
llm = ChatOllama(
    model="llama3.1:8b",  # 사용할 모델을 지정합니다.
)

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.

또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.
쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

만약, 조건별로 안내 내용이 다른 경우
1차 응대 (양해멘트 or 1차 안내 등) + 정보 확인 멘트로 대본을 구성하면 됩니다.
정보 확인 멘트는 "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다." 입니다.
문서의 아래에 각 조건별 대응 방법을 기술해 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.2 이하로 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

prompt = ChatPromptTemplate.from_template(template)

# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)

In [41]:
print("query: ", query)
print("=" * 80)
answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:
    print(ans + '.')

query:  택배사에서 상품을 분실했다고 알림 뜨는데 이거 어떻게 처리해주실 거에요?
Answer : **대본**
불편을 드려 죄송합니다.
 주문하신 상품의 분실 사례입니다.
 택배사에서는 분실 사례에 대한 재배송 및 환불 등 처리를 하실 수 있습니다.
 그러나 이 경우에는 판매자와의 계약된 택배사 확인이 필요하니, 정확한 상담을 위해 주문하신 주문 번호를 알려주시면 확인 후 안내해 드릴게요.


**대응 조건**
1.
 **배송 전 취소**: 상품 준비 중 상태에서 취소가 된 경우, 아직 발송되지 않은 제품의 배송 취소를 할 수 있습니다.
 다만 판매자가 이미 상품을 발송했다면 배송 취소는 불가능하니 택배기사에게 수취거부를 부탁해 주세요.

2.
 **배송 후 취소**: 배송 상태가 [배송완료]로 나타나지만 실제로 상품을 받지 못했다면 배송 완료된 시점의 로켓배송은 대리수령 장소를 확인하거나 마이쿠팡 > 주문목록 > 배송조회 > 배송 사진 확인하실 수 있습니다.
 판매자 배송 상품일 경우에는 문자내용 및 대리수령 가능한 곳을 확인한 후, 로케이션 > 주문목록 > 상품선택 > [택배 배송기사에게 전화하기]로 연락하여 택배기사와 상담해 주시고 택배사의 경우는 배송조회를 통해 안내해 드릴게요.

3.
 **상품 분실**: 택배사에서 상품 분실 알림을 보내셨는데, 정확한 상담을 위해 주문하신 주문 번호를 알려주시면 확인 후 안내해 드릴게요.


위 대본과 조건에 따라 사용자의 답변에 맞춰 안내와 해결까지 도움드리겠습니다.
.
